# FITS
## Flexible Image Transport System

- Es un formato que se diseñó en la decada de los 80
- El paper original es [Wells et al. 1979](https://ui.adsabs.harvard.edu/abs/1979ipia.coll..445W) y [Wells et al 1981](https://ui.adsabs.harvard.edu/abs/1981A%26AS...44..363W/abstract)

<center><img src="images/screenshot_wells1979.png" height="150" alt="Captura paper 1979"/></center>

- Principalmente comenzó a utilizarse para almacenar imágenes CCD
- La idea era estandarizar un formato de transferencia de información astronómica (obvio)
- Y la segunda idea era un almacenamiento a largo plazo de los datos: **_once FITS, always FITS_**

<center><img src="images/screenshot_IAUFITSWG2018_intro.png" height="350" alt="Intro paper 2018"/></center>

### Ventajas y vigencia 

- Posee características que aún prevalecen en la actualidad gracias a las extensiones del formato 
    - Almacena n-arrays, mas propiamente *n-cubes* (espaciado regular)
    - Almacena tablas 2-dimensionales, con filas y columnas
    - Almacena metadatos de forma jerarquica
    - Acepta encapsular multiples conjuntos de datos
    - Es ASCII y se puede abrir y ver que tiene super facil
    - ...

### Descripción del formato

- Descripciones hay por muchos lados, nos guiamos por el documento standar:
    - [IAU FITS WG. 2018](https://fits.gsfc.nasa.gov/fits_standard.html) 
    - Al estar definido en documentos el formato es perdurable y no depende de ningun software
- Vamos por la version 4.0 del formato (2016)

<center><img src="images/FITS_version_tables.png" height="150" alt="Versiones paper 2018"/></center>

- Algo de la historia tecnica del formato
<center><img src="images/milestoneFits_table.png" height="150" alt="Historia tecnica paper 2018"/></center>

- Se utilizan cabeceras `HDU` (o *Header Data Units*) que informan:
    - La estructura del archivo `PrimaryHDU`
    - La estructura de un bloque de datos tipo imagen `ImageHDU`
    - La estructura de un bloque de datos tipo tabla `TableHDU` o bien `BinTableHDU`
- En general los HDU informan cuantos bytes tiene cada bloque y eso los hace 
extensibles agregando nuevos HDU
    - Tienen formato _key: value_ y antiguamente tenian limitaciones en el largo de las llaves
    - Historicamente el maximo eran 36 llaves, de largo total 80 caracteres, y se llamaban _cards_
    - Vamos a ver algunos de los valores necesarios de un HDU
    

- Headers minimo requerido
<center><img src="images/mandatory_primaryHDUcards.png" height="350" alt="Header Primario minimo"/></center>

- Headers de ejemplo
<center><img src="images/minimal_primaryHDU.png" height="150" alt="Header Primario minimo"/></center>

<center><img src="images/minimal_extensionHDU.png" height="150" alt="Header Primario minimo"/></center>


### Utilidades del estandar

- Sistema de expresion de unidades
- Soporta enteros y flotantes, pero no complejos *dentro del estandar*
- Soporta multiplos y submultiplos
- `CHECKSUM` & `DATASUM`: comprobación de integridad de los archivos
- _World Coordinate System_ distortion polynomials
- HEALPix standar transformations

<center><img src="images/basic_unit_expression.png" height="150" alt="Expresiones de unidades basicas"/></center>


<center><img src="images/scaling_multiples.png" height="150" alt="Expresiones de unidades basicas"/></center>


<center><img src="images/BITPIX_values.png" height="150" alt="Expresiones de unidades basicas"/></center>


## Libreria estandar `cfitsio` y `astropy-utils`

La librería base para leer y escribir FITS consiste en CFitsIO.

Posee diversos componentes:
- Libreria de subrutinas de Fortran y C
- Soporte de Checksum para integridad de datos
- Libreria de compresion _fpack_ 
- Visor de imagenes `fv`

Como obtenerla:

```bash
sudo apt install libcfitsio8 libcfitsio-bin python3-fitsio ftools-fv 
```

Además en `astropy` se ofrecen utilidades para la línea de comandos que estan buenas:
```bash
sudo apt install python-astropy astropy-utils
```

#### Demo en terminal

```bash
carbonphoenix ~ > fits
fits2bitmap  fitscopy     fitshdr      fitsmd5      fitstopnm    
fits2table   fitscut      fitsheader   fitsort      fitsverify   
fitscheck    fitsdiff     fitsinfo     fitspng      
```

Se prueba:
- Abrir una imagen con un `cat`, `more`, y `fitsinfo`, `fitshdr`, `fitsheader`
- Chequear `CHECKSUM` con `fitsverify` y validar con `fitsmd5`
- Fijarse en `fitsdiff`, `fitsort`
- Transformar en `png` con `fitspng`, `fitstopnm` `fits2bitmap`
- Hacer una compresion con `fpack`

Los comandos a ver (aprox):

```bash
cat eso085-030-001.fit
more eso085-030-001.fit
fitsinfo eso085-030-001.fit
fitshdr eso085-030-001.fit
fitsheader eso085-030-001.fit
fitsverify eso085-030-001.fit
fitsmd5 esto085-030-001.fit
fitsdiff eso085-030-001.fit eso085-030-003.fit
fitspng eso085-030-001.fit
fitstopnm eso085-030-001.fit > test.pnm
fits2bitmap eso085-030-001.fit
fpack -v eso085-030-001.fit
```

Existe posibilidad de acceder a las distintas unidades con `[ ]`:
```bash
fitspng calexp_01220258-r-R10-S11-det031.fits[1]
```

* Nota sobre `fpack`:
    * La compresion que implementa es de tipo Rice Compression y es muy util
    * Puede reducir en un 6x el espacio en disco casi sin perdida.
    * La perdida es funcion del tipo de pixel, y del ruido en las imagenes, cualquier distribucion complicada es dificil de comprimir
   
<center><img src="images/rice_factor_fpack.png" height="150" alt="Expresiones de unidades basicas"/></center>

<center><img src="images/compressionratio_vs_noise.png" height="150" alt="Expresiones de unidades basicas"/></center>

### DS9

```bash
sudo apt install saods9
```

DS9 es una herramienta de visualizacion de imágenes
- Puede abrir una extension en un frame a la vez

```bash
ds9 eso085-030-003.fit 
```

- Acepta muchas opciones y en realidad esto tiene muchisima funcionalidad

```bash
ds9 eso085-030-003.fit -scale mode zscale -scale asinh -scale limits 0 5 -smooth yes -smooth function tophat -smooth radius 1 
```

### fv

`fv` es otra herramienta para inspeccionar las imagenes
- Puede abrir una extension en un frame a la vez

```bash
fv eso085-030-003.fit 
```

Es muy poco usada

- Usualmente uno llama `imagen` a cualquier FITS file, y la extension de los archivos varia: .FITS, .FIT, .fits, .fit
- Es muy variado como los distintos instrumentos guardan datos en formato FITS, esto no es _standard_
- El estandar especifica muchisimos detalles para las extensiones tipo tabla binaria y tabla

### `astropy.io.fits`

La libreria `astropy` posee utilidades para crear, leer y modificar archivos siguiendo formato FITS

In [ ]:
# modulos que necesitamos
import numpy as np
from astropy.io import fits

In [ ]:
# Algun que otro path para cargar una imagen o almacenar
from pathlib import Path
images = Path('./images')

Podemos abrir un archivo FITS como una imagen con

In [ ]:
hdulist = fits.open(images / 'eso085-030-001.fit')

In [ ]:
print(hdulist.info())

Como vemos se obtiene la misma informacion que en la linea de comandos.   

Se pueden abrir con contexto, como un `with open as...` y cierra el archivo, sino necesitamos ejecutar `hdulist.close()`

Para acceder a las diferentes unidades se usa igual que una coleccion:

In [ ]:
fits_image_filename = fits.util.get_testdata_filepath('test0.fits')
hdulist = fits.open(fits_image_filename)
print(hdulist.info())

In [ ]:
print(repr(hdulist[0].header))

In [ ]:
hdulist[1].header

In [ ]:
hdulist[1].header['BITPIX']

In [ ]:
hdulist[1].header[2:10]

In [ ]:
# los datos se acceden con el atributo `data`
print(hdulist[0].data)

In [ ]:
print(hdulist[1].data)

In [ ]:
# podemos continuar con el tutorial de astropy
# ya que contiene aproximadamente todo lo que 
# se puede hacer con este modulo
... 

Uno puede modificar un archivo existente:

In [ ]:
hdulist = fits.open(images / 'eso085-030-001.fit', mode='update')

In [ ]:
hdulist[0].header

In [ ]:
hdulist[0].header['NOTES  '] = 'Modified by Bruno ;)'

In [ ]:
hdulist[0].header[-5:]

In [ ]:
# hdulist.writeto('modified.fits')
hdulist.flush()

Para crear uno nuevo necesitamos si o si crear una instancia de `PrimaryHDU`

In [ ]:
n = np.arange(100.0)
hdu = fits.PrimaryHDU(n)

n2 = np.random.random(size=(30, 30))
hdu2 = fits.ImageHDU(n2)

hdul = fits.HDUList([hdu, hdu2])

In [ ]:
hdul.writeto('new1.fits')

Hay mucho mas para ver, por ejemplo como abrir imagenes grandes o MEF, tambien como crear una tabla binaria (un formato *muy eficiente*).

Sin embargo lo que mas me gusta son los atajos:

In [ ]:
header = fits.getheader(images / 'eso085-030-001.fit')
data = fits.getdata(images / 'eso085-030-001.fit')
notas = fits.getval(images / 'eso085-030-001.fit', 'OBJECT  ', 0)

In [ ]:
print(notas)

In [ ]:
header['TELESCOP'] = 'TORITOS'
fits.update(images / 'eso085-030-001.fit', data, header, ext=0)